# Listing skills for each job category

**Goal:** given a DataFrame with a column for skills and a column for job categories (e.g., business services, sales & marketing, writing & translation), create a map from each category to all skills required for that category.

# Example input DataFrame

![Example of starting DataFrame](skills-categories-example.png)

# Build example DataFrame

In [1]:
import pandas as pd

In [2]:
df_example = pd.DataFrame({'Skills':
                   [
                        'human resources;leadership;business development',
                        'creative thinking;marketing communications',
                        'microsoft excel', 
                        'marketing communications;animated video', 
                        'thesis;environmental management;industry analysis', 
                        'creative vision;branding'
                   ],
                   'Categories':
                    [
                        'business services;sales & marketing',
                        'writing & translation;business services;sales & marketing',
                        'administrative support;business services',
                        'design & multimedia;sales & marketing',
                        'writing & translation;business services',
                        'design & multimedia;business services;sales & marketing'
                    ]
                  })

In [3]:
# Reorder columns if needed
df = df_example[['Skills', 'Categories']]
df

,Skills,Categories
0,human resources;leadership;business development,business services;sales & marketing
1,creative thinking;marketing communications,writing & translation;business services;sales ...
2,microsoft excel,administrative support;business services
3,marketing communications;animated video,design & multimedia;sales & marketing
4,thesis;environmental management;industry analysis,writing & translation;business services
5,creative vision;branding,design & multimedia;business services;sales & ...


# Try 1


The most naive approach is simply to create a new DataFrame that maps each category to a list of skills mentioned in the original DataFrame. This doesn't involve any actual machine-learning, but it may be good enough (and fast enough) to be rerun each time the original dataframe is updated.